# Consulta interactiva de valores de jugadores

### Objetivo
En este notebook creamos un **asistente de consulta** que permite buscar el valor de mercado de un jugador y obtener:

- Valor oficial de Transfermarkt (última temporada disponible).
- Valor estimado por el modelo **XGBoost**.
- Diferencia absoluta y porcentual.
- Posibilidad de consultar cualquier jugador dentro del dataset.

Esto convierte al proyecto en una **herramienta interactiva**, más allá de los análisis estáticos.


In [1]:
# ========================================
# 1. Importar librerías
# ========================================
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ========================================
# 2. Cargar dataset
# ========================================
df = pd.read_csv("../data/processed/dataset_model.csv")
print("[i] Dataset cargado:", df.shape)
df.head()


[i] Dataset cargado: (6849, 275)


,Player,Squad,Pos,Age,Born,season,Competition,player_key,squad_key,season_key,...,Nation_us USA,Nation_uy URU,Nation_uz UZB,Nation_ve VEN,Nation_wls WAL,Nation_xk KVX,Nation_za RSA,Nation_zm ZAM,Nation_zw ZIM,market_value_log
0,Anthony Khelifa,Ajaccio,DF,16.0,2005.0,2022-2023,big5,anthony khelifa,ajaccio,2022-2023,...,False,False,False,False,False,False,False,False,False,0.000000
1,Ben Hamed Touré,Ajaccio,FW,18.0,2003.0,2022-2023,big5,ben hamed toure,ajaccio,2022-2023,...,False,False,False,False,False,False,False,False,False,0.000000
2,Benjamin Leroy,Ajaccio,GK,33.0,1989.0,2022-2023,big5,benjamin leroy,ajaccio,2022-2023,...,False,False,False,False,False,False,False,False,False,13.304687
3,Bevic Moussiti-Oko,Ajaccio,FW,27.0,1995.0,2022-2023,big5,bevic moussiti-oko,ajaccio,2022-2023,...,False,False,False,False,False,False,False,False,False,13.458837
4,Clément Vidal,Ajaccio,"DF,MF",22.0,2000.0,2022-2023,big5,clement vidal,ajaccio,2022-2023,...,False,False,False,False,False,False,False,False,False,14.220976


### Preparación del dataset
- Se definen las variables predictoras (X) y el target (y).
- Se aplica one-hot encoding.
- Se entrena nuevamente el modelo XGBoost para poder hacer consultas dinámicas.


In [2]:
# Variables
X = df.drop(columns=["market_value_log","market_value_in_eur"])
y = df["market_value_log"]

# One-hot encoding
X_encoded = pd.get_dummies(X, drop_first=True)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

# Modelo
xgb = XGBRegressor(
    n_estimators=500, learning_rate=0.1, max_depth=6,
    random_state=42, n_jobs=-1
)
xgb.fit(X_train, y_train)

print("[i] Modelo entrenado listo para consultas.")


[i] Modelo entrenado listo para consultas.


### Función de consulta
La siguiente función permite ingresar el **nombre de un jugador** y obtener:
- Valor oficial de Transfermarkt.
- Valor estimado por el modelo.
- Diferencia absoluta y porcentual.


In [29]:
# ========================================
# Función de consulta
# ========================================
def consultar_jugador(nombre, df_actual, df_forecast, model, season_actual="2024-2025", season_future="2025-2026"):
    """
    Consulta el valor de un jugador en Transfermarkt vs modelo,
    e incluye predicción de la temporada siguiente.
    Permite búsqueda parcial, sin importar mayúsculas/tildes.
    """
    import unidecode
    
    # Normalizar texto
    def normalizar(txt):
        return unidecode.unidecode(str(txt).lower())
    
    # Filtrar jugador actual
    df_actual["nombre_norm"] = df_actual["Player"].apply(normalizar)
    nombre_norm = normalizar(nombre)
    
    jugador = df_actual[(df_actual["nombre_norm"].str.contains(nombre_norm)) & (df_actual["season"] == season_actual)]
    
    if jugador.empty:
        return pd.DataFrame([{
            "Jugador": nombre,
            "Error": f"No encontrado en {season_actual}"
        }])
    
    player_name = jugador["Player"].values[0]
    valor_transfer = jugador["market_value_in_eur"].values[0]
    
    # Features para predicción actual
    X_jugador = pd.get_dummies(jugador.drop(columns=["market_value_log","market_value_in_eur","nombre_norm"]), drop_first=True)
    X_jugador = X_jugador.reindex(columns=X_encoded.columns, fill_value=0)
    
    pred_log = model.predict(X_jugador)
    pred_actual = np.expm1(pred_log)[0]
    
    # Predicción para la temporada futura
    jugador_future = df_forecast[df_forecast["Player"] == player_name]
    if not jugador_future.empty:
        pred_future = jugador_future["predicted_value"].values[0]
    else:
        pred_future = None
    
    # Tabla estética
    data = {
        "Jugador": [player_name],
        "Temporada actual": [season_actual],
        "Transfermarkt (€)": [f"€{valor_transfer:,.0f}"],
        "Modelo (actual)": [f"€{pred_actual:,.0f}"],
        "Diferencia": [f"€{pred_actual - valor_transfer:,.0f}"],
        "% Diferencia": [f"{(pred_actual - valor_transfer) / valor_transfer * 100:.2f}%"],
        f"Predicción {season_future}": [f"€{pred_future:,.0f}" if pred_future else "N/A"]
    }
    
    return pd.DataFrame(data)

In [ ]:
# ========================================
# Definir datasets base
# ========================================

# df_actual = jugadores con temporada actual (2024/2025)
df_actual = df[df["season"] == "2024-2025"].copy()

# df_forecast = predicciones de TODOS los jugadores (2025/2026)

X_all = X_encoded.loc[df_actual.index]
pred_log = xgb.predict(X_all)
pred_valores = np.expm1(pred_log)

df_forecast = df_actual.copy()
df_forecast["predicted_value"] = pred_valores
df_forecast["season"] = "2025-2026"

### Ejemplos de consulta
Probar con jugadores famosos de la temporada 2024/2025.


In [ ]:
# Probar consultas

jugadores = ["Kylian Mbappe", "Haaland", "Yamal", "Lewandowski", "Musiala"]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

resultados = pd.concat([consultar_jugador(j, df_actual, df_forecast, xgb) for j in jugadores], ignore_index=True)

import IPython.display as disp
disp.display(resultados)

,Jugador,Temporada actual,Transfermarkt (€),Modelo (actual),Diferencia,% Diferencia,Predicción 2025-2026
0,Kylian Mbappé,2024-2025,"€160,000,000","€84,020,592","€-75,979,408",-47.49%,"€171,985,568"
1,Erling Haaland,2024-2025,"€200,000,000","€55,754,448","€-144,245,552",-72.12%,"€121,228,968"
2,Lamine Yamal,2024-2025,"€180,000,000","€39,080,284","€-140,919,716",-78.29%,"€65,666,480"
3,Robert Lewandowski,2024-2025,"€15,000,000","€8,314,516","€-6,685,484",-44.57%,"€14,707,751"
4,Jamal Musiala,2024-2025,"€140,000,000","€57,466,492","€-82,533,508",-58.95%,"€116,565,192"
